In [11]:
import paramiko
import datetime
import pandas as pd
import numpy as np
import os

print(datetime.datetime.now())

2020-06-08 14:28:32.468542


In [2]:
host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "vgn5UucsUNHL4n9R" #hard-coded
username = "biglots_data" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [9]:
list_remote_files=sftp.listdir("./")
list_remote_files=['med_med.txt','high_high.txt']

In [15]:
local_dir_1="/home/jian/BigLots/M1_audience/audience_from_client_"+str(datetime.datetime.now().date())+"/"
local_dir_2="./audience_from_client_"+str(datetime.datetime.now().date())+"/"


try:
    os.stat(local_dir_1)
except:
    os.mkdir(local_dir_1)
    
try:
    os.stat(local_dir_2)
except:
    os.mkdir(local_dir_2)


for base_name in list_remote_files:
    sftp.get(base_name,local_dir_1+base_name)
    sftp.get(base_name,local_dir_2+base_name)
sftp.close()
transport.close()

In [10]:
list_remote_files

['med_med.txt', 'high_high.txt']

In [29]:
df_med_med=pd.read_table(local_dir_2+"med_med.txt",dtype=str,sep="\t")
df_high_high=pd.read_table(local_dir_2+"high_high.txt",dtype=str,sep="\t")

In [30]:
df_med_med['med_med']=1
df_high_high['high_high']=1
df_med_med=df_med_med[['hashed_email_address','med_med']].drop_duplicates()
df_high_high=df_high_high[['hashed_email_address','high_high']].drop_duplicates()
print(df_med_med.shape)
print(df_high_high.shape)

df_both=pd.merge(df_med_med,df_high_high,on="hashed_email_address",how="outer").fillna(0)
print(df_both.shape,df_both['hashed_email_address'].nunique())
df_both=df_both.rename(columns={"hashed_email_address":"Email_Address"})
df_both['med_med']=df_both['med_med'].astype(int)
df_both['high_high']=df_both['high_high'].astype(int)
df_both=df_both.rename(columns={"med_med":"BL_Rewards_2020_Q2_med_med","high_high":"BL_Rewards_2020_Q2_high_high"})


(3485389, 2)
(1787174, 2)
(5272549, 3) 5272549


In [38]:
df_both.head(3)

,Email_Address,BL_Rewards_2020_Q2_med_med,BL_Rewards_2020_Q2_high_high
0,a76413fc9d1689f35a2361b8daff43e4ebf3e1dfd63f39...,1,0
1,de20dc7e6faa632b5fed14bd78a2cd22123915baf36365...,1,0
2,81d194f92c28b5e0c95fc9b87791bbaade920ea185040c...,1,0


In [40]:
df_both.to_csv(path_or_buf="./"+"BL_SHA256EMAIL_Rewards2020Q2_BLMatrics_"+str(datetime.datetime.now().date())+".txt",
                         sep='\t',
                         encoding='utf-8',
                         line_terminator="\n",
                         header=True,
                         index=False)